<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/master_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Project 3: Team Decision Makers***
## Jesse Kranyak &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Siriesha Mandava
## Mohamed Altoobli &nbsp;&nbsp;   Jeffery Boczkaja

# Skin Lesion Classification Project
This project focuses on classifying skin lesions using deep learning techniques. The dataset used for training and testing the model is the ISIC 2019 dataset, which consists of images of various skin lesions along with associated metadata.

## Step 1: Initial setup and Library installation
Colab requires many libraries to be installed for use everytime you run a new notebook!

In [1]:
!pip install tensorflow
!pip install isic-cli
!pip install isic-cli
!pip install kaggle
!pip install imblearn
!pip install optuna
!pip install tqdm
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 2.2 MB/s eta 0:00:00


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import requests
import seaborn as sns
import shutil
import tensorflow as tf

from datetime import datetime
from google.colab import drive
from imblearn.over_sampling import SMOTE
from IPython.display import Image
from optuna.integration import TFKerasPruningCallback
from pathlib import Path
from PIL import Image

from sklearn.metrics import auc, classification_report, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import SimpleImputer

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling2D,
                                     Input, MaxPooling2D, concatenate)
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.utils import Sequence, plot_model, to_categorical

from tqdm import tqdm


## Step 2: Query the Dataset
In this step, we interact with the ISIC API to query the dataset directly. We use specific API endpoints to perform operations such as listing available images, retrieving image metadata, and downloading images. This section covers setting up API requests, managing data retrieval, and handling query parameters to filter the dataset based on criteria such as diagnosis, image type, or other metadata.

In [2]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=FX7JwnPRV1QTePPgPQ8zzLI3u362HR&code_challenge=P4zXBIfazY65I2l-ppCXQsSgzUR3wW3jcaRz7rHVLtQ&code_challenge_method=S256
enter the code shown in your browser: RKWDN4MFVruyytmS32P7pNXwMQTx9x
Success!


In [3]:
!isic collection list

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None            │
│ 64  │ Challenge 2018: Task 1-2: Test                │ True   │ True   │ True   │ None    

In [5]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


Be sure to have kaggle API key ready to go in .json file. Place it into the 'content' folder.

####Necessary Mdels and Datasets are available in Git Repo folder:

#####saved_models_for_final_project

In [23]:
# Specify the current path of kaggle.json file
current_path = '/content/kaggle.json'

# Desired path where the Kaggle API expects the kaggle.json file
desired_path = '/root/.kaggle/kaggle.json'

if os.path.exists(current_path):
    os.makedirs(os.path.dirname(desired_path), exist_ok=True)
    os.rename(current_path, desired_path)

    # Set the file's permissions to avoid a permissions error
    os.chmod(desired_path, 0o600)
else:
    print(f"Error: '{current_path}' does not exist. Please upload the file.")



Error: '/content/kaggle.json' does not exist. Please upload the file.


find the data set on kaggle

Comment out if running and already have downloaded images

In [7]:
!kaggle datasets download -d andrewmvd/isic-2019


100% 9.09G/9.10G [01:11<00:00, 118MB/s]
100% 9.10G/9.10G [01:11<00:00, 137MB/s]


In [8]:
!unzip -q isic-2019.zip


## Step 3: Data Preparation and Preprocessing DICOM Images
This step involves handling the metadata for the ISIC 2019 dataset, which which will be used for training, testing, and validation data. The processes highlighted in this step are crucial for understanding the dataset's structure and preparing it for subsequent analysis.

IMPORTANT NOTE we removed the DICOM image sets from this project for ease of processing on our colab wallets...




In [ ]:
# Load the metadata
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

# Display the first few rows of each DataFrame
metadata.head()


In [43]:
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
ground_truth

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Load the CSV files
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

image_dir = Path('/content/ISIC_2019_Training_Images')
ground_truth['image_path'] = ground_truth['image'].apply(lambda x: image_dir / f"{x}.jpg")

# Merge the ground_truth with metadata if necessary
full_metadata = pd.merge(ground_truth, metadata, on='image', how='left')  # Adjust 'on' parameter as needed
full_metadata

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,image_path,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,55.0,anterior torso,NaN,female
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,anterior torso,NaN,female
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,60.0,upper extremity,NaN,female
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,upper extremity,NaN,male
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,80.0,posterior torso,NaN,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,85.0,head/neck,BCN_0003925,female
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,65.0,anterior torso,BCN_0001819,male
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,70.0,lower extremity,BCN_0001085,male
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007325...,55.0,palms/soles,BCN_0002083,female


In [45]:
# Display the image paths for the first few entries
print(full_metadata['image_path'].head())


0    /content/ISIC_2019_Training_Images/ISIC_000000...
1    /content/ISIC_2019_Training_Images/ISIC_000000...
2    /content/ISIC_2019_Training_Images/ISIC_000000...
3    /content/ISIC_2019_Training_Images/ISIC_000000...
4    /content/ISIC_2019_Training_Images/ISIC_000000...
Name: image_path, dtype: object


In [46]:
# Correct the base path in 'image_path' column
correct_base_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

full_metadata['image_path'] = full_metadata['image'].apply(lambda x: f"{correct_base_path}/{x}.jpg")

# Verify the correction by printing the first few entries again
print(full_metadata['image_path'].head())


0    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
1    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
2    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
3    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
4    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
Name: image_path, dtype: object


In [47]:
img_path = full_metadata['image_path']

In [48]:
# Fill missing values for 'age_approx' with its median
full_metadata['age_approx'].fillna(full_metadata['age_approx'].median(), inplace=True)

# For categorical data, fill missing values with 'unknown'
full_metadata['anatom_site_general'].fillna('unknown', inplace=True)
full_metadata['sex'].fillna('unknown', inplace=True)


In [49]:
full_metadata = pd.get_dummies(full_metadata, columns=['sex'])


In [52]:
# Handle missing values in 'lesion_id'
full_metadata['lesion_id'].fillna('unknown', inplace=True)

# Convert booleans to integers
bool_columns = ['sex_female', 'sex_male', 'sex_unknown']
full_metadata[bool_columns] = full_metadata[bool_columns].astype(int)

# One-hot encoding of 'anatom_site_general'
full_metadata = pd.get_dummies(full_metadata, columns=['anatom_site_general'])

# Ensure all floating point columns are float32
float_columns = full_metadata.select_dtypes(include=['float64']).columns
full_metadata[float_columns] = full_metadata[float_columns].astype('float32')

# Convert boolean columns from one-hot encoding to integers
for col in full_metadata.columns:
    if full_metadata[col].dtype == 'bool':
        full_metadata[col] = full_metadata[col].astype(int)

# Normalize the 'age_approx' column (Optional: based on model requirements)
full_metadata['age_approx'] = full_metadata['age_approx'].astype('float32') / 100

full_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25331 entries, 0 to 25330
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   image                                25331 non-null  object 
 1   MEL                                  25331 non-null  float32
 2   NV                                   25331 non-null  float32
 3   BCC                                  25331 non-null  float32
 4   AK                                   25331 non-null  float32
 5   BKL                                  25331 non-null  float32
 6   DF                                   25331 non-null  float32
 7   VASC                                 25331 non-null  float32
 8   SCC                                  25331 non-null  float32
 9   UNK                                  25331 non-null  float32
 10  image_path                           25331 non-null  object 
 11  age_approx                  

In [58]:
full_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25331 entries, 0 to 25330
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   image                                25331 non-null  object 
 1   MEL                                  25331 non-null  float32
 2   NV                                   25331 non-null  float32
 3   BCC                                  25331 non-null  float32
 4   AK                                   25331 non-null  float32
 5   BKL                                  25331 non-null  float32
 6   DF                                   25331 non-null  float32
 7   VASC                                 25331 non-null  float32
 8   SCC                                  25331 non-null  float32
 9   UNK                                  25331 non-null  float32
 10  image_path                           25331 non-null  object 
 11  age_approx                  

In [135]:
# Save the cleaned and encoded DataFrame to a CSV file
full_metadata.to_csv('/content/full_metadata.csv', index=False)

##Step 4: Experimenting on the TTS&V

### Splitting Data into Training, Validation, and Testing Sets

The dataset is initially split into training and temporary sets, with the temporary set reserved for further division into validation and test sets. This method ensures that the model can be trained extensively, validated accurately, and finally tested to evaluate its performance on unseen data.

##results before image input augmentation part 2
###Test 1, no modification 68% acc score
###Test 2, resampling 67% acc score

| Class | Count    |
|-------|----------|
| MEL   | 20651.0  |
| NV    | 6957.0   |
| BCC   | 25309.0  |
| AK    | 77387.0  |
| BKL   | 30227.0  |
| DF    | 261561.0 |
| VASC  | 261561.0 |
| SCC   | 99679.0  |


###Test 3, weighted resampling 69.1%

| Class | Samples  |
|-------|----------|
| MEL   | 373.0    |
| NV    | 130.0    |
| BCC   | 504.0    |
| AK    | 7682.0   |
| BKL   | 644.0    |
| DF    | 268194.0 |
| VASC  | 268194.0 |
| SCC   | 15448.0  |

###Test 4 , 62% acc
New class counts after resampling w communism:

| Class | Samples |
|-------|---------|
| MEL   | 191.0   |
| NV    | 191.0   |
| BCC   | 191.0   |
| AK    | 191.0   |
| BKL   | 191.0   |
| DF    | 191.0   |
| VASC  | 191.0   |
| SCC   | 191.0   |

###Test 5, lets try a different kind of distributoin and classify the critically important (malignant) lesions with more weight:

| Class | Samples |
|-------|---------|
| MEL   | 1910.0  |
| NV    | 191.0   |
| BCC   | 1910.0  |
| AK    | 191.0   |
| BKL   | 191.0   |
| DF    | 191.0   |
| VASC  | 191.0   |
| SCC   | 524.0   |

####Not much happening with this adjustment, will leave it weighted, with punishment on malignant classes and start hypertuning





In [17]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.utils import resample

# # Assuming 'full_metadata' is your complete dataset
# X = full_metadata.drop(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
# y = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']]

# # Split the data into training and testing sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Calculate class weights
# class_weights = 1 / y_train.sum()
# class_weights /= class_weights.sum()

# # Calculate the target number of samples based on weighted average
# weighted_samples = (class_weights * y_train.sum().sum()).astype(int)

# # Create new DataFrame for the balanced dataset
# X_train_resampled = pd.DataFrame(columns=X_train.columns)
# y_train_resampled = pd.DataFrame(columns=y_train.columns)

# # Resampling
# for column in y_train.columns:
#     filter_mask = y_train[column] == 1
#     X_class = X_train[filter_mask]
#     y_class = y_train[filter_mask]

#     num_samples = len(X_class)
#     target_samples = weighted_samples[column]
#     if num_samples > 0:
#         if num_samples < target_samples:
#             X_class_resampled, y_class_resampled = resample(X_class, y_class,
#                                                             replace=True,  # Sample with replacement
#                                                             n_samples=target_samples,  # Match the target samples
#                                                             random_state=42)
#         else:
#             X_class_resampled, y_class_resampled = resample(X_class, y_class,
#                                                             replace=False,
#                                                             n_samples=target_samples,
#                                                             random_state=42)

#         # Append resampled data back to the overall dataset
#         X_train_resampled = pd.concat([X_train_resampled, X_class_resampled], axis=0)
#         y_train_resampled = pd.concat([y_train_resampled, y_class_resampled], axis=0)
#     else:
#         print(f"No instances to resample for class '{column}'")

# # Shuffle the dataset to mix up class order (important for training)
# X_train_resampled = X_train_resampled.sample(frac=1, random_state=42)
# y_train_resampled = y_train_resampled.loc[X_train_resampled.index]

# print("New class counts after resampling:\n", y_train_resampled.sum())


In [131]:
# Assuming 'full_metadata' is your complete dataset and properly loaded
X = full_metadata.drop(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
y = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']]

# Define class weights based on your specification
weights = {
    'MEL': 10.0, 'NV': 1.0, 'BCC': 10.0, 'AK': 1.0, 'BKL': 1.0,
    'DF': 10.0, 'VASC': 1.0, 'SCC': 10.0
}

# Calculate class totals and total samples for weight adjustments
class_totals = y.sum()
total_samples = y.sum().sum()

# Compute weights inversely proportional to class frequencies
class_weights = total_samples / (len(class_totals) * class_totals)

# Adjust weights according to your specific weights
adjusted_weights = {key: weights[key] for key in weights}

# Split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Output the class weights for reference
print("New class counts after resampling:\n", y_train_resampled.sum())

New class counts after resampling:
 MEL     1910.0
NV       191.0
BCC     1910.0
AK       191.0
BKL      191.0
DF       191.0
VASC     191.0
SCC      524.0
dtype: float32


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.utils import resample

# # Determine the count of each class in the training data
# class_counts = y_train.sum()

# # Find the minimum class size (size of the smallest class) and ensure it's an integer
# min_class_size = int(class_counts.min())

# # Modify the resampling code to undersample the majority classes
# X_train_resampled = pd.DataFrame(columns=X_train.columns)
# y_train_resampled = pd.DataFrame(columns=y_train.columns)

# for column in y_train.columns:
#     filter_mask = y_train[column] == 1
#     X_class = X_train[filter_mask]
#     y_class = y_train[filter_mask]

#     num_samples = len(X_class)

#     # Check if the current class is a majority class and it has more samples than the minimum
#     if num_samples > min_class_size:
#         # Apply undersampling
#         X_class_resampled, y_class_resampled = resample(X_class, y_class,
#                                                         replace=False,
#                                                         n_samples=min_class_size,  # Reducing to the size of the smallest class
#                                                         random_state=42)
#     else:
#         # If not a majority class, keep as is
#         X_class_resampled, y_class_resampled = X_class.copy(), y_class.copy()

#     # Append resampled data back to the overall dataset
#     X_train_resampled = pd.concat([X_train_resampled, X_class_resampled], axis=0)
#     y_train_resampled = pd.concat([y_train_resampled, y_class_resampled], axis=0)

# # Shuffle the dataset to mix up class order (important for training)
# X_train_resampled = X_train_resampled.sample(frac=1, random_state=42)
# y_train_resampled = y_train_resampled.loc[X_train_resampled.index]

# print("New class counts after resampling:\n", y_train_resampled.sum())


In [76]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


(20264, 16) (20264, 8)
(2533, 16) (2533, 8)
(2534, 16) (2534, 8)


## Step 5: First Model Preparation and Training

We will use a Dual Input Generator we custom built for our first model and the following steps will prep, then train our model.

### Dual Input Data Generator

The `DualInputGenerator` class is a custom data generator for handling datasets that include both image paths and associated metadata. It is designed to work with Keras/TensorFlow, enabling efficient batch processing which is crucial for training deep learning models on large datasets. Here's a breakdown of its functionality:

- **Initialization (`__init__`)**: This method sets up the generator with all necessary parameters, including image paths, metadata, and labels. It also initializes the batch size, image size, and whether the dataset should be shuffled during training to introduce randomness into the training process.

- **Preprocessing (`preprocess_image`)**: A helper function to read and preprocess images. It converts images to the appropriate size and scale ([224x224] in this case) and normalizes pixel values to the range [0, 1].

- **Length Calculation (`__len__`)**: This method calculates how many batches are in the dataset, which is used by Keras during training to determine the number of steps per epoch.

- **Batch Generation (`__getitem__`)**: This method retrieves a batch of data by processing the images, metadata, and labels. It loads and preprocesses the images specified by the batch indexes, extracts the corresponding metadata, and gathers the labels. The function returns a list containing two arrays (images and metadata) and the batch of labels.

- **Epoch End Handling (`on_epoch_end`)**: If shuffling is enabled, this method shuffles the indexes after each epoch to ensure that the model does not see the same sequence of batches every epoch, helping the model to generalize better.

This structured approach ensures that the model receives properly formatted and preprocessed data for each training step, facilitating effective learning and performance improvement.


###Updated input generator to provide code for more data augmentation on images, using

1.   flip: left and right
2.   brightness & contrast shifting

####Not a large effect noticed in this type of augmentation on our specific model



In [132]:
class DualInputGenerator(Sequence):
    def __init__(self, image_paths, metadata, labels, batch_size, img_size=(224, 224), shuffle=True):
        self.image_paths = image_paths
        self.metadata = metadata
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def preprocess_image(self, file_path):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = img / 255.0  # Normalize to [0, 1]

        # Apply data augmentation
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, max_delta=0.2)
        img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
        # Additional augmentations can be added here

        return img

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = (index + 1) * self.batch_size
        end_idx = min(end_idx, len(self.image_paths))
        current_batch_size = end_idx - start_idx

        X_images = np.empty((current_batch_size, *self.img_size, 3))
        X_metadata = np.empty((current_batch_size, self.metadata.shape[1]))
        y = np.empty((current_batch_size, self.labels.shape[1]), dtype=int)

        for i, idx in enumerate(range(start_idx, end_idx)):
            img_path = self.image_paths[idx]
            img = self.preprocess_image(img_path)
            X_images[i, ] = img
            X_metadata[i, ] = self.metadata[idx]
            y[i, ] = self.labels[idx]

        return [X_images, X_metadata], y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


### Initializing Data Generators for Training, Validation, and Testing

To ensure our model is trained, validated, and tested with data that is efficiently loaded and preprocessed, we initialize three instances of the `DualInputGenerator` class:

- **Training Generator (`train_gen`)**: This generator is configured with training data paths, metadata, and labels. It is responsible for feeding the training data into the model in batches of 32, ensuring that each batch is shuffled to promote model generalization.

- **Validation Generator (`val_gen`)**: Similar to the training generator, but using the validation dataset. This generator provides data for evaluating the model during the training process, allowing us to monitor the model's performance and make adjustments if needed without seeing the test data.

- **Testing Generator (`test_gen`)**: Finally, the testing generator is set up using the test dataset to evaluate the model's performance after training has been completed. This step is crucial for assessing how well the model is likely to perform on unseen real-world data.

Each generator uses the `image_paths`, `metadata`, and `labels` from their respective subsets of the data, ensuring that the model receives all necessary inputs for making predictions during each phase of the training and evaluation process.



In [136]:
# Initialize the training generator with image size parameter
train_gen = DualInputGenerator(
    image_paths=X_train['image_path'].values,
    metadata=X_train.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_train.values,
    batch_size=32,
    img_size=(224, 224)
)

# Initialize the validation generator with image size parameter
val_gen = DualInputGenerator(
    image_paths=X_val['image_path'].values,
    metadata=X_val.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_val.values,
    batch_size=32,
    img_size=(224, 224)
)

# Initialize the test generator with image size parameter
test_gen = DualInputGenerator(
    image_paths=X_test['image_path'].values,
    metadata=X_test.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_test.values,
    batch_size=32,
    img_size=(224, 224)
)

# Example line to check if the image paths from X_test are being accessed correctly
print(X_test['image_path'].values)


['/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0031599.jpg'
 '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0057178.jpg'
 '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0060330.jpg'
 ...
 '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0056152.jpg'
 '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0070938.jpg'
 '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0057966.jpg']


### Dual Input Neural Network Architecture

The neural network architecture is designed to handle two types of input: images and metadata. Here's a breakdown of the model architecture and the role of each component:

#### Image Input Branch
- **Image Input**: The model starts with an image input layer that accepts images of shape (224, 224, 3).
- **Convolution and Pooling Layers**: Following the input, the model includes two sets of convolutional layers (`Conv2D`) and max pooling layers (`MaxPooling2D`). Each convolutional layer is followed by batch normalization (`BatchNormalization`), which helps to accelerate the training process and stabilize the learning environment by normalizing the activations.
- **Flattening and Dense Layer**: After extracting and pooling features through convolutions, the data is flattened (`Flatten`) and passed through a dense layer with ReLU activation, which is again batch normalized.

#### Metadata Input Branch
- **Metadata Input**: This branch begins with an input for metadata features, shaped dynamically based on the number of metadata features (`num_metadata_features`).
- **Dense Layers and Normalization**: It includes dense layers (`Dense`) with ReLU activation, interspersed with batch normalization to ensure the model learns effectively from the structured data.

#### Combining Branches
- **Concatenation**: The outputs of the image and metadata branches are combined into a single vector (`concatenate`), allowing the model to learn from both image features and metadata simultaneously.
- **Final Dense Layers**: The combined data is then passed through additional dense layers, including a dropout layer (`Dropout`) to prevent overfitting, culminating in a softmax output layer (`Dense`) that classifies the images into one of nine diagnostic categories.



In [137]:
num_metadata_features = X_train.drop(columns=['image', 'image_path', 'lesion_id']).shape[1]
print("Number of metadata features:", num_metadata_features)


Number of metadata features: 13


In [138]:
num_metadata_features = 13

# Image input branch
image_input = Input(shape=(224, 224, 3), name='image_input')
x = Conv2D(32, (3, 3), activation='relu')(image_input)
# Add batch normalization here
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
# Add batch normalization here
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
image_branch = Dense(128, activation='relu')(x)
image_branch = BatchNormalization()(image_branch)

# Metadata input branch
metadata_input = Input(shape=(num_metadata_features,), name='metadata_input')
y = Dense(32, activation='relu')(metadata_input)
# Normalize after the first dense layer
metadata_branch = BatchNormalization()(y)
metadata_branch = Dense(64, activation='relu')(metadata_branch)
# Normalize before combining
metadata_branch = BatchNormalization()(metadata_branch)

# Combine the outputs of the two branches
combined = concatenate([image_branch, metadata_branch])
z = Dense(256, activation='relu')(combined)
 # Adding dropout for regularization
z = Dropout(0.2232357393141235)(z)
# Adjust the number of units to match the number of classes
output = Dense(8, activation='softmax')(z)

# Create the model
model = Model(inputs=[image_input, metadata_input], outputs=output)

# Adjust the learning rate if testing indicates it's beneficial
# Start with your last setting and adjust based on performance
learning_rate = 1e-5


##Augmentaion of Automated hyperparameter optimization ☄

After numerous trials we are unable to break the low 70% accuracies, we will try some hypertuning...

### How Optuna Chooses the Optimizer:
In your objective function, you specify a categorical distribution for the optimizer using `trial.suggest_categorical()`. This function tells Optuna to randomly choose one of the provided options for each trial.

### Optuna's Optimization Algorithms:
Optuna uses several algorithms to optimize the search for the best hyperparameters:

- **Tree-structured Parzen Estimator (TPE):** A default and commonly used approach that models the search space using Bayesian optimization.
- **CMA-ES:** An evolutionary algorithm suitable for optimizing in continuous spaces, which can be more effective when many trials are possible.
- **Random Search and Grid Search:** Simpler methods that are sometimes used, particularly random search in the early phases of a study.


###Extra method set here to reduce time;

def objective_and_fit_wrapper(trial):

Parallelize Computations: Structuring our model and computations to maximize parallelism, allowing the TPU to efficiently utilize its cores. This lets us run more epochs faster. We also applied pruning to the studies and that will end the early identified losers more rapidly.

In [128]:
# # Assuming 'full_metadata' is your complete dataset and properly loaded
# X = full_metadata.drop(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
# y = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']]

# # Split the data into training, validation, and testing sets (these are not augmented)
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

class DualInputGenerator(Sequence):
    def __init__(self, image_paths, metadata, labels, batch_size, img_size=(224, 224), shuffle=True):
        self.image_paths = image_paths
        self.metadata = metadata
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def preprocess_image(self, file_path):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, self.img_size)
        img = img / 255.0
        return img

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = (index + 1) * self.batch_size
        end_idx = min(end_idx, len(self.image_paths))
        current_batch_size = end_idx - start_idx
        X_images = np.empty((current_batch_size, *self.img_size, 3))
        X_metadata = np.empty((current_batch_size, self.metadata.shape[1]))
        y = np.empty((current_batch_size, self.labels.shape[1]), dtype=int)
        for i, idx in enumerate(range(start_idx, end_idx)):
            img_path = self.image_paths[idx]
            img = self.preprocess_image(img_path)
            X_images[i, ] = img
            X_metadata[i, ] = self.metadata[idx]
            y[i, ] = self.labels[idx]
        return [X_images, X_metadata], y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

def build_model(learning_rate, dropout_rate, num_metadata_features):
    image_input = Input(shape=(224, 224, 3), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    image_branch = Dense(128, activation='relu')(x)
    image_branch = BatchNormalization()(image_branch)

    metadata_input = Input(shape=(num_metadata_features,), name='metadata_input')
    y = Dense(32, activation='relu')(metadata_input)
    metadata_branch = BatchNormalization()(y)
    metadata_branch = Dense(64, activation='relu')(metadata_branch)
    metadata_branch = BatchNormalization()(metadata_branch)

    combined = concatenate([image_branch, metadata_branch])
    z = Dense(256, activation='relu')(combined)
    z = Dropout(dropout_rate)(z)
    output = Dense(8, activation='softmax')(z)

    model = Model(inputs=[image_input, metadata_input], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def objective_and_fit(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    model = build_model(learning_rate, dropout_rate, 13)
    train_gen = DualInputGenerator(X_train['image_path'].values, X_train.drop(columns=['image', 'image_path', 'lesion_id']).values, y_train.values, batch_size)
    val_gen = DualInputGenerator(X_val['image_path'].values, X_val.drop(columns=['image', 'image_path', 'lesion_id']).values, y_val.values, batch_size)
    # Add TFKerasPruningCallback to enable pruning at each epoch
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        TFKerasPruningCallback(trial, 'val_accuracy')  # Monitor validation accuracy for pruning
    ]
    history = model.fit(train_gen, epochs=3, validation_data=val_gen, callbacks=callbacks, verbose=1)
    max_val_accuracy = np.max(history.history['val_accuracy'])
    return max_val_accuracy

# Set up parallelism for Optuna
def objective_and_fit_wrapper(trial):
    return objective_and_fit(trial)

if __name__ == '__main__':
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,  # Number of trials to run before pruning starts
        n_warmup_steps=2,    # Number of steps to run before starting to apply prune logic
        interval_steps=1     # Interval steps to apply pruning
    )

    study = optuna.create_study(direction='maximize', pruner=pruner)

    # Uncomment the line below to run the optimization
    #study.optimize(objective_and_fit_wrapper, n_trials=50)

    print("Best hyperparameters: ", study.best_trial.params)



[I 2024-04-21 20:38:23,214] A new study created in memory with name: no-name-986a4a84-32b5-4693-b627-5f7a69ed52c8


Epoch 1/3
1266/1266 [==============================] - 452s 355ms/step - loss: 1.1587 - accuracy: 0.5881 - val_loss: 1.0641 - val_accuracy: 0.6250
Epoch 2/3
1266/1266 [==============================] - 449s 354ms/step - loss: 1.0308 - accuracy: 0.6233 - val_loss: 1.0507 - val_accuracy: 0.6191
Epoch 3/3
1266/1266 [==============================] - 451s 356ms/step - loss: 0.9874 - accuracy: 0.6379 - val_loss: 1.0772 - val_accuracy: 0.6230


[I 2024-04-21 21:00:55,401] Trial 0 finished with value: 0.625 and parameters: {'learning_rate': 0.0028012345263178262, 'dropout_rate': 0.16355387148358058, 'batch_size': 16}. Best is trial 0 with value: 0.625.


Epoch 1/3
633/633 [==============================] - 383s 601ms/step - loss: 1.1478 - accuracy: 0.5985 - val_loss: 0.9860 - val_accuracy: 0.6535
Epoch 2/3
633/633 [==============================] - 383s 605ms/step - loss: 0.9100 - accuracy: 0.6739 - val_loss: 0.9244 - val_accuracy: 0.6709
Epoch 3/3
633/633 [==============================] - 383s 605ms/step - loss: 0.7479 - accuracy: 0.7365 - val_loss: 0.8903 - val_accuracy: 0.6804


[I 2024-04-21 21:20:05,345] Trial 1 finished with value: 0.6803797483444214 and parameters: {'learning_rate': 7.879710412375101e-05, 'dropout_rate': 0.14240551142447977, 'batch_size': 32}. Best is trial 1 with value: 0.6803797483444214.


...

Epoch 1/3
633/633 [==============================] - 401s 628ms/step - loss: 1.1125 - accuracy: 0.5996 - val_loss: 1.0653 - val_accuracy: 0.6274
Epoch 2/3
633/633 [==============================] - 401s 633ms/step - loss: 0.9256 - accuracy: 0.6656 - val_loss: 0.9953 - val_accuracy: 0.6535
Epoch 3/3
633/633 [==============================] - 404s 637ms/step - loss: 0.8128 - accuracy: 0.7031 - val_loss: 0.9343 - val_accuracy: 0.6646


[I 2024-04-22 01:52:37,613] Trial 15 finished with value: 0.6645569801330566 and parameters: {'learning_rate': 0.0003222053459025417, 'dropout_rate': 0.10052000502500325, 'batch_size': 32}. Best is trial 8 with value: 0.6827531456947327.


Epoch 1/3
393/633 [=================>............] - ETA: 2:20 - loss: 1.1823 - accuracy: 0.5881

[W 2024-04-22 01:56:31,265] Trial 16 failed with parameters: {'learning_rate': 0.0004278505111587838, 'dropout_rate': 0.2534668355578268, 'batch_size': 32} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-128-7466004b9a42>", line 98, in objective_and_fit_wrapper
    return objective_and_fit(trial)
  File "<ipython-input-128-7466004b9a42>", line 92, in objective_and_fit
    history = model.fit(train_gen, epochs=3, validation_data=val_gen, callbacks=callbacks, verbose=1)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-package

KeyboardInterrupt: 

#### Model Compilation
- Now we can implement the hypertuned parameters discovered in the previous steps... fingers crossed we get a better output.  This process is automated, as the runtime of the above step is long.
- The model was originally compiled with the Adam optimizer, using a learning rate of 1e-4. The loss function used is categorical crossentropy, suitable for multi-class classification tasks, and accuracy is used as the metric to evaluate model performance.

This dual-input setup allows the model to leverage both detailed image data and accompanying metadata, aiming to improve diagnostic accuracy compared to using images or metadata alone.


In [ ]:
# Retrieve the best hyperparameters
best_params = study.best_trial.params
best_learning_rate = best_params['learning_rate']
best_optimizer_name = best_params['optimizer']

# Choose the optimizer based on the best hyperparameters found
if best_optimizer_name == 'adam':
    optimizer = Adam(learning_rate=best_learning_rate)
else:
    optimizer = RMSprop(learning_rate=best_learning_rate)

# Compile the model with the best hyperparameters
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)


###Manual save in case colab boots us...
run these 2 cells instead of the 2 cells above to save 15 - 96 hours

In [ ]:
# best parameters from tuning process
# best_params = {
#     'learning_rate': 0.00029207045028036043,
#     'optimizer': 'rmsprop',
#     'batch_size': 64,
#     'objective_value': 0.6776107549667358
# }

# # hypertested at .72 loss .74 acc
# best_params = {
#     'learning_rate': 0.00195497006883763,
#     'optimizer': 'rmsprop',
#     'batch_size': 16,
#     'objective_value': 0.6507120132446289
# }

#
# best_params = {
#     'learning_rate': 0.00033474734484637854,
#     'optimizer': 'rmsprop',
#     'batch_size': 64,
#     'objective_value': 0.6673259735107422
# }

best_params = {
    'learning_rate': 0.00012617133871374036,
    'optimizer': 'rmsprop',
    'batch_size': 32,
    'objective_value': 0.6673259735107422
}

# Save the parameters to disk
with open('/content/drive/My Drive/best_hyperparams.pkl', 'wb') as f:
    pickle.dump(best_params, f)




In [142]:
# Load the saved parameters
with open('/content/drive/My Drive/best_hyperparams.pkl', 'rb') as f:
    best_params = pickle.load(f)

# Assume `model` is your pre-existing model that you've already defined and compiled somewhere in your code

# Configure the optimizer based on the loaded parameters
if best_params['optimizer'] == 'adam':
    optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    optimizer = RMSprop(learning_rate=best_params['learning_rate'])
else:
    raise ValueError("Unsupported optimizer: {}".format(best_params['optimizer']))

# Set the new optimizer to the model
optimizer = Adam(learning_rate=0.00012617133871374036)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


### Configuring Callbacks for Model Training

To enhance the training process and ensure optimal model performance, we use a combination of callbacks in TensorFlow. These callbacks are set to monitor the training process and make decisions at the end of each epoch. Here's a breakdown of each callback used:

#### EarlyStopping
- **Purpose**: Prevents overfitting by halting the training if the model's validation loss does not improve for a specified number of consecutive epochs.
- **Configuration**:
  - `monitor='val_loss'`: Watches the validation loss metric for changes.
  - `patience=2`: Allows the training to continue for 2 epochs without improvements in validation loss, we will run more later with a higher patience level.
  - `verbose=1`: Enables logging for when the training is stopped early.

#### ModelCheckpoint
- **Purpose**: Saves the model in its current state after each epoch, but only if the model's performance (based on the monitored metric) has improved.
- **Configuration**:
  - `filepath='best_model.keras'`: Specifies the location and filename where the best model version will be saved.
  - `monitor='val_loss'`: Monitors the validation loss for improvements.
  - `save_best_only=True`: Ensures that the model is saved only when its validation loss is at its lowest point seen so far.
  - `verbose=1`: Provides detailed logs when the model is saved.

These callbacks are essential tools for managing long training sessions effectively. They help conserve resources by stopping training when additional epochs would not lead to improvements (`EarlyStopping`) and by ensuring that only the best model version is saved (`ModelCheckpoint`), thus simplifying deployment and further evaluation.


In [143]:
checkpoint_path = "/content/drive/My Drive/your_model_directory/model-{epoch:02d}-{val_accuracy:.2f}.hdf5"


In [144]:
callbacks = [
    EarlyStopping(monitor='val_loss',
                  mode='min',
                  verbose=3,
                  # Allows some room for loss to decrease again
                  patience=10,
                  # Stops training if the loss goes above 0.2
                  baseline=0.25,
                  restore_best_weights=True
                  ),


    ModelCheckpoint(filepath='best_model.keras',
                    monitor='val_loss',
                    save_best_only=True,
                    verbose=1
                    )
]

### Training the Model
#### AND Re-Training, the first model resulted in 70% this is after going back to do futher augmentation

To train the dual-input model, we use the `model.fit` method provided by TensorFlow. This method executes the training cycle across a specified number of epochs, allowing for detailed monitoring and adjustments during the training process. Here's a breakdown of each parameter used in the `model.fit` call:

- **train_gen**: The training data generator, which supplies batches of image and metadata inputs along with the corresponding labels.
- **validation_data**: The validation data generator used to evaluate the model at the end of each epoch, helping monitor its performance on unseen data.
- **epochs**: Sets the total number of training cycles the model will undergo. In this case, the model is set to train for 10 epochs.
- **steps_per_epoch**: Specifies the number of batch steps to complete one epoch. This is set to the total number of batches available in the training generator (`len(train_gen)`), ensuring that each sample is used once per epoch.
- **validation_steps**: Determines the number of batch steps used for validating the model, set to the length of the validation generator.
- **callbacks**: Includes the `EarlyStopping` and `ModelCheckpoint` callbacks configured previously. These enhance the training process by:
  - **EarlyStopping**: Automatically stops training when the validation loss ceases to decrease, preventing overfitting.
  - **ModelCheckpoint**: Saves the best version of the model based on validation loss, ensuring that only the most accurate model is retained.

This configuration ensures that the model is not only trained but also validated effectively, with checkpoints saved automatically and training potentially halted early if no further gains are observed. This approach optimizes both the efficiency and efficacy of the model training process.



In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    # Include callbacks for early stopping and model checkpointing
    callbacks=callbacks
)


Epoch 1/10
633/633 [==============================] - ETA: 0s - loss: 1.2084 - accuracy: 0.5752
Epoch 1: val_loss improved from inf to 1.03564, saving model to best_model.keras
633/633 [==============================] - 402s 631ms/step - loss: 1.2084 - accuracy: 0.5752 - val_loss: 1.0356 - val_accuracy: 0.6250
Epoch 2/10
633/633 [==============================] - ETA: 0s - loss: 1.0193 - accuracy: 0.6308
Epoch 2: val_loss improved from 1.03564 to 0.97243, saving model to best_model.keras
633/633 [==============================] - 404s 637ms/step - loss: 1.0193 - accuracy: 0.6308 - val_loss: 0.9724 - val_accuracy: 0.6428
Epoch 3/10
633/633 [==============================] - ETA: 0s - loss: 0.9468 - accuracy: 0.6569
Epoch 3: val_loss improved from 0.97243 to 0.91466, saving model to best_model.keras
633/633 [==============================] - 403s 637ms/step - loss: 0.9468 - accuracy: 0.6569 - val_loss: 0.9147 - val_accuracy: 0.6610
Epoch 4/10
633/633 [==============================] - ET

In [ ]:
# Evaluate the model on the test set
results = model.evaluate(test_gen, steps=len(test_gen))
test_loss, test_accuracy = results

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


## Step 6: Model Summary and Visualization

The `model.summary()` method outputs the architecture of the model, listing each layer, its type, output shape, and number of parameters. This helps understanding the depth and complexity of the model's structure. Additionally, the `plot_model` function is used to generate a graphical representation of the model, which includes the shapes and names of each layer, offering a visual reference of the model's design.

### Generating and Evaluating Predictions

After training, predictions are generated for the test set using the `model.predict` method. This function computes the model's output predictions. The `np.argmax` function is then applied to these predictions to convert the model outputs from probabilities to explicit class predictions.

### Collecting True Classes and Generating Confusion Matrix

To evaluate the accuracy of our model's predictions, it is necessary to compare these predicted classes against the true classes from the test set. This comparison is facilitated through the collection of true class labels directly from the test generator. Subsequently, a confusion matrix is generated using the `confusion_matrix` function. This matrix visualizes the performance of a classification model, showing the actual versus predicted classifications, which helps in identifying how well the model is performing with respect to different classes. The `seaborn.heatmap` function is then used to plot the confusion matrix, providing a color-coded visualization of the results, which makes it easier to interpret the model's accuracy and misclassifications.



In [ ]:
model.summary()

# Generate a plot of the model
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)


In [ ]:
# Generate predictions using the test generator
predictions = model.predict(test_gen, steps=len(test_gen))

# Obtain the predicted classes by taking the argmax of the predictions array
predicted_classes = np.argmax(predictions, axis=1)

# test_gen should also be outputting the true classes in the same order as the predictions:
# We need to collect all true classes from the generator (in the same order)
true_classes = []
for _, labels in test_gen:
    true_classes.extend(np.argmax(labels, axis=1))
    if len(true_classes) >= len(predictions):
        break  # Stop once we have all the labels we need

# Generate the confusion matrix
cm = confusion_matrix(true_classes[:len(predictions)], predicted_classes)

print(cm)

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

###Summary of Confusion Matrix
Classes 3, 4, 5, 6, and 7 show a particular need for augmentation due to low true positives and high confusion with other classes. Class 0 also needs attention due to being frequently confused with other high-incidence classes. These focused augmentations aim to enhance the model's ability to distinguish these classes more clearly, which should improve overall accuracy and reduce misclassifications.

we will try and address this through resampling methods. Ideally we would jsut augment the data.

####SEE section of repository labled experimental_code for our attempt at creating GAN images.

## Step 7: Further Model Evaluation and Misclassification Analysis

The process starts by loading a previously trained model from disk using `load_model`. This model is then used to evaluate its performance on unseen data provided by a test generator. The evaluation consists of several steps:

#### Loading the Model
The model is loaded from a saved file, ensuring that the most effective version of the model, as determined by prior validations, is used for evaluations.

#### Fetching Predictions and Labels
A custom function `get_predictions_and_labels` is employed to iterate over the test generator. This function fetches batches of data (both images and metadata) and corresponding labels, then uses the model to predict each batch. Predictions and true labels are accumulated from all batches, allowing for comprehensive evaluation.

#### Calculating Confusion Matrix and Classification Report
Once predictions are gathered they are converted from softmax probabilities to class predictions using `np.argmax`. With predictions and true labels in hand, a confusion matrix is generated to visually assess the model's performance across different classes. This matrix highlights which classes are being confused with others, providing insight into potential biases or weaknesses in the model.

A classification report is also generated to provide key metrics for each class, such as precision, recall, and F1-score. This report helps in understanding the model's accuracy and identifying classes that might require more focus during further model training.

#### Visualizing Misclassified Examples
Finally, the script identifies and displays a set of misclassified images along with their true and predicted labels. This visualization is crucial for diagnosing what might be causing the errors, potentially guiding further data collection, augmentation strategies, or model adjustments.

Each of these steps plays a role in evaluating the model's real-world applicability and robustness, ensuring that the model not only performs but also meets practical expectations.

Lets just run it a few more times and see if it can get any better before it gets any worse...

In [ ]:
# Load the previously saved model
model = load_model('/content/best_model.keras')

# Define your callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=1),
    ModelCheckpoint(filepath='/content/best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

# Path where the checkpoint will be saved
model_dir = '/content/drive/My Drive/Colab Models'
# Generate a unique identifier based on the current date and time
unique_identifier = datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_filepath = os.path.join(model_dir, f'best_model_{unique_identifier}.keras')


# Continue training with generators and include callbacks
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    # Adjust the total epochs as needed
    epochs=20,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=callbacks
)


In [ ]:
# Load the model
model = load_model('/content/best_model.keras')

# Function to fetch data and labels from the generator
def get_predictions_and_labels(generator):
    predictions = []
    true_labels = []
    for _ in range(len(generator)):
        # Getting the next batch of data
        (images, metadata), labels = generator.__getitem__(_)
        # Predict this batch
        batch_predictions = model.predict([images, metadata])
        # Store predictions and labels
        predictions.append(batch_predictions)
        true_labels.append(labels)

    # Concatenate all batches
    predictions = np.vstack(predictions)
    true_labels = np.vstack(true_labels)

    return predictions, true_labels

# Using the test generator to get predictions and labels
predictions, true_labels = get_predictions_and_labels(test_gen)

# Convert softmax probabilities to class predictions
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(true_labels, axis=1)

# Calculate confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
print('Confusion Matrix:\n', cm)

# Generate a classification report
print('\nClassification Report:\n', classification_report(true_classes, predicted_classes))

# Identify misclassified examples
misclassified_indices = np.where(predicted_classes != true_classes)[0]
print(f'Total misclassified samples: {len(misclassified_indices)}')

# Assuming you want to visualize some misclassified images:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
for i, ax in enumerate(axes.flatten()):
    if i < len(misclassified_indices):
        idx = misclassified_indices[i]
        img = test_gen.image_paths[idx]  # Assuming image paths are stored here
        true_label, pred_label = true_classes[idx], predicted_classes[idx]
        ax.imshow(plt.imread(img))  # Reading the image from path
        ax.set_title(f'True: {true_label}, Pred: {pred_label}')
        ax.axis('off')
plt.tight_layout()
plt.show()


### Evaluation of Classifier Performance Using ROC Curves

To evaluate the performance of the classifier on a per-class basis, the Receiver Operating Characteristic (ROC) curves and the Area Under the Curve (AUC) are computed. These metrics are crucial for assessing the model's ability to distinguish between classes. Here's a breakdown of the steps involved in this evaluation:

#### Preparing Labels and Predictions
Firstly, the true labels are converted to a 1-dimensional array if they are not already in this format, ensuring they can be directly compared with the predicted probabilities. Predictions are checked to ensure they contain a probability for each class.

#### Computing ROC Curves and AUC Values
For each class, the ROC curve is calculated by:
- Isolating the prediction probabilities for that class.
- Creating a binary outcome array for the class, where the class of interest is labeled `1` and all others `0`.
- Using the `roc_curve` function from `sklearn.metrics` to compute the true positive rate (TPR) and false positive rate (FPR) at various threshold settings.
- Calculating the AUC value to quantify the overall ability of the model to discriminate between positive and negative classes for each specific class.

#### Plotting the ROC Curves
Each class's ROC curve is plotted with a unique color, and the AUC score is displayed in the legend to provide a visual and numerical representation of classifier performance. The diagonal line represents a random classifier's performance for comparison.

- **X-axis**: False Positive Rate (FPR) — represents the proportion of negative data points that are mistakenly considered positive.
- **Y-axis**: True Positive Rate (TPR) — represents the proportion of actual positives correctly identified.

The area under each curve (AUC) provides a single measure of overall performance regardless of the classification threshold. The closer the AUC is to 1, the better the model is at predicting positive classes as positive and negative classes as negative. Values closer to 0.5 suggest no discriminative ability, equivalent to random guessing.

This detailed analysis helps in identifying which classes the model performs well on and which ones might require further tuning, potentially guiding further data augmentation, additional training, or algorithm adjustments.


In [ ]:
# Ensure that true_labels is a 1D array of class indices
if len(true_labels.shape) > 1:
    true_labels = np.argmax(true_labels, axis=1)

# Ensure predictions are probabilities
if predictions.shape[1] == len(np.unique(true_labels)):
    print("Predictions appear to be properly formatted.")

n_classes = len(np.unique(true_labels))
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    # Isolate the probabilities for the current class
    class_probs = predictions[:, i]
    # Create a binary outcome for this class
    class_true = (true_labels == i).astype(int)
    fpr[i], tpr[i], _ = roc_curve(class_true, class_probs)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = plt.cm.viridis(np.linspace(0, 1, n_classes))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve by class')
plt.legend(loc="lower right")
plt.show()



##Step 8: "Teachers: The Architects of Tomorrow's... Model"

### Knowledge Distillation:
Uses our current model (the "teacher") to train a simpler model (the "student"). The student model learns to mimic the teacher model's behavior while often generalizing better on unseen data due to its simplicity.

### Fine-tuning with Regularization:
Load the weights of the best model and continue training with increased regularization (L1/L2 regularization) to force the model to learn more robust features, just because these students are in a BOOTCAMP.

###And TA's help

lets add in another layer of model prediction using visualBert

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
import tensorflow_hub as hub

# Paths to your data
image_dir = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
metadata_csv_path = "/content/full_metadata.csv"

# Load metadata
metadata = pd.read_csv(metadata_csv_path)

# Prepare the image dataset
image_paths = metadata['image_path'].apply(lambda x: f"{image_dir}/{x}")
image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to the size expected by VisualBERT
    image = image / 255.0  # Normalize pixel values
    return image

image_dataset = image_dataset.map(preprocess_image)

# Load the pre-trained VisualBERT model
visual_bert_hub_url = "https://tfhub.dev/google/visualbert/1"  # Assuming this URL is correct
visual_bert_layer = hub.KerasLayer(visual_bert_hub_url, trainable=True)

# Drop unnecessary columns from metadata after preparing image paths
metadata.drop(columns=['image', 'image_path', 'lesion_id'], inplace=True)

# Prepare the metadata features
metadata_features = metadata.values
metadata_dataset = tf.data.Dataset.from_tensor_slices(metadata_features)

# Combine image and metadata into a single dataset
combined_dataset = tf.data.Dataset.zip((image_dataset, metadata_dataset))

# Define the model
input_image = Input(shape=(224, 224, 3), name="input_image")
# Adjust the shape according to actual metadata features
input_metadata = Input(shape=(13,), name="input_metadata")

# VisualBERT processing
visual_bert_output = visual_bert_layer(input_image)

# Concatenate VisualBERT output with metadata
combined_features = Concatenate()([visual_bert_output, input_metadata])

# Classification head
classification_output = Dense(8, activation='softmax')(combined_features)

# Final model
model = Model(inputs=[input_image, input_metadata], outputs=classification_output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Define training procedure
# Assuming you have a way to split the combined_dataset into training and validation datasets
# Define training procedure with prefetching
train_dataset = combined_dataset.take(int(len(combined_dataset)*0.8)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = combined_dataset.skip(int(len(combined_dataset)*0.8)).batch(32).prefetch(tf.data.AUTOTUNE)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

In [ ]:
results = model.evaluate(val_dataset)
print("Loss, Accuracy:", results)

In [ ]:
model.save('/content/best_ta_model.keras')

In [ ]:
import optuna
from optuna.pruners import MedianPruner

def objective(trial):
    # Hyperparameters to tune
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.3, 0.7)
    l1_value = trial.suggest_loguniform('l1_value', 1e-5, 1e-3)
    l2_value = trial.suggest_loguniform('l2_value', 1e-4, 1e-2)

    # Model building function with tuned hyperparameters
    def build_tuned_student_model():
        from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
        from keras.models import Model
        from keras.optimizers import Adam
        from keras.regularizers import l1_l2
        from keras.callbacks import EarlyStopping, ModelCheckpoint

        # Define the model architecture
        image_input = Input(shape=(224, 224, 3), name='image_input')
        x = Conv2D(32, (3, 3), activation='relu')(image_input)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        image_branch = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=l1_value, l2=l2_value))(x)

        metadata_input = Input(shape=(13,), name='metadata_input')
        y = Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1_value, l2=l2_value))(metadata_input)
        metadata_branch = BatchNormalization()(y)

        # Combine the image and metadata branches
        combined = concatenate([image_branch, metadata_branch])
        z = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1_value, l2=l2_value))(combined)
        z = Dropout(dropout_rate)(z)
        output = Dense(8, activation='softmax')(z)

        # Compile the model
        student_model = Model(inputs=[image_input, metadata_input], outputs=output)
        student_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

        return student_model

    # Compile and fit the model
    student_model = build_tuned_student_model()
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=2, verbose=1),
        ModelCheckpoint(filepath='/content/student_best_model_tuned.keras', monitor='val_loss', save_best_only=True, verbose=1)
    ]
    history = student_model.fit(train_gen, validation_data=val_gen, epochs=6, steps_per_epoch=len(train_gen), validation_steps=len(val_gen), callbacks=callbacks)

    # Report the best validation loss to the trial
    best_val_loss = min(history.history['val_loss'])
    return best_val_loss

# Create a study object with a median pruner
study = optuna.create_study(direction='minimize', pruner=MedianPruner(n_startup_trials=10,
                                                                      n_warmup_steps=3,
                                                                      interval_steps=1
                                                                      ))
with ThreadPoolExecutor(max_workers=4) as executor:
    study.optimize(objective, n_trials=50, n_jobs=4)

# uncomment to run hypertuning
study.optimize(objective, n_trials=50)



In [ ]:
# Load the teacher model
teacher_model = load_model('/content/best_model.keras')

# Define the student model with increased regularization
def build_student_model(input_shape_image=(224, 224, 3), num_metadata_features=13, num_classes=8):
    # Image input branch - Reuses the same architecture as the teacher model
    image_input = Input(shape=input_shape_image, name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    image_branch = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)

    # Metadata input branch - Simplified version of the teacher model
    metadata_input = Input(shape=(num_metadata_features,), name='metadata_input')
    y = Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(metadata_input)
    metadata_branch = BatchNormalization()(y)

    # Combine the outputs of the two branches
    combined = concatenate([image_branch, metadata_branch])
    z = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(combined)
    z = Dropout(0.5)(z)
    output = Dense(num_classes, activation='softmax')(z)

    # Create the student model
    student_model = Model(inputs=[image_input, metadata_input], outputs=output)

    # Compile the student model
    student_model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    return student_model

# Build the student model
student_model = build_student_model()

# Define callbacks for the student model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),
    ModelCheckpoint(filepath='/content/student_best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

# Initialize the training generator
train_gen = DualInputGenerator(
    image_paths=X_train['image_path'].values,
    metadata=X_train.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_train.values,
    batch_size=32
)

# Initialize the validation generator
val_gen = DualInputGenerator(
    image_paths=X_val['image_path'].values,
    metadata=X_val.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_val.values,
    batch_size=32
)

# Fit the student model -
history = student_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=callbacks
)



In [ ]:
# Define the path where the model should be saved
model_path = '/content/best_student_model.keras'

# Save the model
student_model.save(model_path)

print(f"Model saved successfully at {model_path}")


In [ ]:
test_loss, test_accuracy = student_model.evaluate(test_gen)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

####These students requires a lot of training....
but it looks like they are improving over time.


In [ ]:

# Set the filepath for the saved model
filepath = '/content/best_student_model.keras'

# Define the ModelCheckpoint callback to save the best model
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch'
)

# Define the EarlyStopping callback to stop training when no improvement is seen
early_stopping = EarlyStopping(
    monitor='val_loss',
    # Adjust the patience parameter based on your tolerance for waiting,
    # this one is set high because im going to bed...
    patience=10,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# List of callbacks
callbacks = [checkpoint, early_stopping]

# Continue training the model
history_continued = student_model.fit(
    train_gen,
    validation_data=val_gen,
    # We set a higher number since training will stop automatically
    epochs=50,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=callbacks
)


In [ ]:
test_loss, test_accuracy = student_model.evaluate(test_gen)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


###Teamwork makes the dream work.

lets see how they perform as a team

In [ ]:
# Assuming models are already loaded; otherwise, load them:
teacher_model = load_model('/content/best_model.keras')
student_model = load_model('/content/best_student_model.keras')
teaching_assistant_model = load_model('/content/best_ta_model.keras')

# Function to evaluate ensemble accuracy
def evaluate_ensemble_accuracy(teacher_model, student_model, test_gen):
    correct_count = 0
    total_count = 0

    # Correctly unpack the data from the generator
    for (images, metadata), labels in test_gen:
        # Predict with both models
        teacher_predictions = teacher_model.predict([images, metadata])
        student_predictions = student_model.predict([images, metadata])

        # Average the predictions (change based on each model's acc results)
        averaged_predictions = (teacher_predictions * 0.8) + (student_predictions * 0.2)

        # Convert probabilities to predicted class index
        predicted_classes = np.argmax(averaged_predictions, axis=1)
        true_classes = np.argmax(labels, axis=1)  # Assuming labels are one-hot encoded

        # Increment correct count
        correct_count += np.sum(predicted_classes == true_classes)
        total_count += labels.shape[0]

    # Calculate accuracy
    accuracy = correct_count / total_count
    return accuracy

# Evaluate ensemble accuracy using the provided test generator
accuracy = evaluate_ensemble_accuracy(teacher_model, student_model, test_gen)
print("Ensemble accuracy: {:.2f}%".format(accuracy * 100))

OK, well not every dream comes true, but we came this far, lets apply the models

In [ ]:
# Source directory containing the files you want to copy
source_dir = '/content/sample_data'

# Target directory on Google Drive
target_dir = '/content/drive/MyDrive/metadata_and_models'

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Copy all files from the source directory to the target directory
for file_name in os.listdir(source_dir):
    shutil.copy(os.path.join(source_dir, file_name), os.path.join(target_dir, file_name))

print("Files copied successfully!")


# Step 8: **The Interface** ⚡

---




User Interface (UI)
A simple web-based UI allows users to upload lesion images, input relevant metadata, and receive a prediction. We will use Gradio as learned in class for this model.

In [ ]:
! pip install gradio --upgrade
! pip install --upgrade gradio
! pip install gradio ai

In [ ]:
import gradio as gr
from gradio import Image, Number, Radio, Dropdown

import numpy as np
from PIL import Image
import os
from PIL import Image as PILImage

import tensorflow as tf
from tensorflow import keras

### Model Loading and Prediction Testing

#### Loading the Pre-trained Model
The script begins by loading a pre-trained model from a specified path. This model is essential for predicting the type of skin lesions based on image data and associated metadata.

#### Model Architecture Overview
Once the model is loaded, `model.summary()` is called to print the architecture of the model. This summary provides insight into the model's layers, their shapes, and parameters, which is crucial for understanding how the model processes input data.

#### Dummy Prediction
To ensure the model is functioning correctly, a dummy input array is created using random data, shaped appropriately for the model's expected input (in this case, 224x224x3 for image data). This array is then used to make a prediction, testing the model's readiness for actual data.

### Image Preprocessing and Prediction Explanation Function

#### Image Preprocessing
The script defines a function `preprocess_image` that takes an uploaded image file, resizes it to fit the model's input requirements, normalizes the pixel values, and expands its dimensions to include a batch size for model input compatibility.

#### Prediction and Explanation Generation
Another function, `predict_and_explain`, is designed to handle the end-to-end process from image and metadata input through to generating a human-readable explanation of the prediction. This function:
- Preprocesses the input image for model prediction.
- Uses the model to predict the lesion type based on the image.
- Maps the prediction to a readable class description.
- Optionally, uses GPT-3 to generate a detailed explanation of the diagnosis, integrating the lesion's metadata for a comprehensive overview.

### User Interface for Skin Lesion Classification

#### Interface Setup
The script utilizes `gr.Interface` from the Gradio library to create an interactive web interface. This interface allows users to upload images of skin lesions and input relevant metadata (age, sex, and anatomical site).

#### Interface Launch
Finally, the interface is launched, making it accessible via a web browser. Users can interact with the model, upload images, input metadata, and receive predictions along with explanations right in the interface.

### Usage and Disclaimer
The interface includes a title and a detailed description, advising users on how to use the tool and noting that predictions should not replace professional medical advice. This ensures users understand the context and limitations of the model predictions.

This comprehensive setup allows for an accessible and user-friendly way to leverage advanced machine learning models for educational and preliminary diagnostic support in dermatology.


In [ ]:
import numpy as np
import tensorflow.keras as keras

# Load the model
model_path = '/content/best_model.h5'
model = keras.models.load_model(model_path)

# Print model summary
model.summary()

# Create a dummy input array for metadata
test_metadata = np.random.random((1, 13))  # Assuming metadata has shape (1, 13)

# Combine image data and metadata
test_input_image = np.random.random((1, 224, 224, 3))  # Assuming image data has shape (1, 224, 224, 3)
test_input = [test_input_image, test_metadata]

# Make a prediction
test_prediction = model.predict(test_input)
print(test_prediction)


###Tri-Model Predictions
##The Sudents & The Teachers 🥇
###...and a little help from TA's
###An Ode to Python AI Coding Bootcamp

**Initial Binary Classification**: Here we use the binary medical classifier, Cancer-Net SCa Model, to determine if a lesion is malignant or benign. This first step allows the system to filter and focus the subsequent predictions.

**Conditional Multiclass Classification**:
- **If Malignant**: Route the input to the student-teacher model but restrict the classification to the malignant classes only. This reduces the complexity of the prediction task for these cases.
- **If Benign**: Proceed with the student-teacher model using all classes or possibly excluding known malignant classes based on the prior binary result.

This conditional approach leverages the strength of both models and optimizes the classification task by reducing the decision space when appropriate, potentially increasing overall accuracy.



In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import gradio as gr

# Load the pre-trained teacher and student models
teacher_model_path = '/content/best_model.keras'
student_model_path = '/content/best_student_model.keras'
teacher_model = load_model(teacher_model_path)
student_model = load_model(student_model_path)

# Load the binary classifier, our TA
# last model test; loss: 0.3760 - accuracy: 0.8446 - val_loss: 0.3826 - val_accuracy: 0.8474

binary_classifier_path = 'content/best_cancer_net_sca_model.keras'
binary_classifier = load_model(binary_classifier_path)

# Dictionary of diagnosis descriptions
diagnosis_descriptions = {
    'MEL': 'Melanoma: a serious form of skin cancer that begins in cells known as melanocytes.',
    'NV': 'Melanocytic nevus: a common type of skin growth that often appears as a small, dark brown spot.',
    'BCC': 'Basal cell carcinoma: a type of skin cancer that most often develops on areas exposed to the sun.',
    'AK': 'Actinic keratosis: a rough, scaly patch on the skin caused by years of sun exposure.',
    'BKL': 'Benign keratosis: a non-cancerous skin condition that appears as a waxy brown, black, or tan growth.',
    'DF': 'Dermatofibroma: a common growth on the skin, usually found on the lower legs, that can be pink, red, or brown.',
    'VASC': 'Vascular lesion: a type of abnormal growth or mark on the skin that is made up of blood vessels.',
    'SCC': 'Squamous cell carcinoma: a common form of skin cancer that develops in the squamous cells.'
}

def preprocess_image(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image = np.array(image)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

def predict_and_explain(image, age, sex, location):
    # Preprocess the image
    image = preprocess_image(image)

    # Encode the sex and location for the student and teacher model
    sex_encoded = [1 if sex == 'Male' else 0, 1 if sex == 'Female' else 0, 1 if sex == 'Other' else 0]
    location_encoded = [1 if location == loc else 0 for loc in ['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other']]

    # Construct metadata for student and teacher models, which seems to require more detailed encoding
    std_teacher_metadata = np.array([[age] + sex_encoded + location_encoded])

    # Construct metadata for binary classifier, adjust this based on your actual model's requirements
    binary_classifier_metadata = np.array([[1 if condition in ["MEL", "melanocytic", "NV", "benign", "concomitant_biopsy"] else 0 for condition in location_encoded]])

    # Binary classification to determine malignancy
    is_malignant = binary_classifier.predict([image, binary_classifier_metadata])[0][0] > 0.5

    # Remaining part of your function
    if is_malignant:
        relevant_classes = ['MEL', 'BCC', 'SCC']  # Malignant classes
        mask = np.isin(np.array(list(diagnosis_descriptions.keys())), relevant_classes)
    else:
        mask = np.ones(len(diagnosis_descriptions), dtype=bool)

    # Predict with both models
    teacher_prediction = teacher_model.predict([image, std_teacher_metadata])[0]
    student_prediction = student_model.predict([image, std_teacher_metadata])[0]
    average_prediction = (teacher_prediction + student_prediction) / 2
    masked_prediction = average_prediction[mask]

    predicted_class = np.argmax(masked_prediction)
    class_labels = np.array(list(diagnosis_descriptions.keys()))[mask]
    predicted_label = class_labels[predicted_class]

    description = diagnosis_descriptions[predicted_label]
    response = f"Predicted Lesion Type: {predicted_label}\nDescription: {description}\nMalignant: {'Yes' if is_malignant else 'No'}"
    return response

iface = gr.Interface(
    fn=predict_and_explain,
    inputs=[
        gr.Image(image_mode='RGB', sources=["upload", "webcam"], type='pil'),
        gr.Number(label="Age"),
        gr.Radio(choices=['Male', 'Female', 'Other'], label="Sex"),
        gr.Dropdown(choices=['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other'], label="Anatomical Site")
    ],
    outputs="text",
    title="Skin Lesion Classifier",
    description="Upload an image of a skin lesion and enter metadata to predict its type. The results provide a preliminary classification and are not a substitute for professional medical advice."
)

iface.launch(debug=True)


# Skin Lesion Classification Project Summary

&emsp;&emsp;This project focused on classifying skin lesions using deep learning techniques with the ISIC 2019 dataset. A central aspect was the development of a dual-input model that integrates both image data and patient metadata, aiming to leverage additional context for improved diagnostic accuracy. TheDualInputGenerator was utilized for handling the simultaneous input of images and metadata during training. This setup facilitated the effective integration of both data types. The training process was fine-tuned using strategies like early stopping and model checkpoints, ensuring excellent model performance without extensive straining the our computational resources. <br>
<br>
&emsp;&emsp;A Gradio interface was deployed to allow user interaction with the model, enabling uploads of skin lesion images and entry of relevant metadata for real-time predictions. This user-friendly interface makes the model accessible to non-experts and provides educational insights into different types of skin lesions, demonstrating the application's potential as a diagnostic aid. <br>
<br>
&emsp;&emsp;The project highlighted the application of mixed data models in medical image analysis, demonstrating how machine learning can be applied in healthcare settings. It also underscored the importance of user interface design in making complex models accessible and useful to a broader audience. There is potential to expand the model to include more diagnostic categories and enhance the interface with more detailed explanations of predictions. Further development could involve integrating AI-driven insights or additional data sources. User testing and feedback will be essential in refining the tool to better meet user needs. The ultimate goal could be to integrate this tool into medical systems for preliminary diagnostics.

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import gradio as gr

# # Load the pre-trained teacher and student models
# teacher_model_path = '/content/best_model.keras'
# student_model_path = '/content/best_student_model.keras'
# teacher_model = load_model(teacher_model_path)
# student_model = load_model(student_model_path)

# # Dictionary of diagnosis descriptions
# diagnosis_descriptions = {
#     'MEL': 'Melanoma: a serious form of skin cancer that begins in cells known as melanocytes.',
#     'NV': 'Melanocytic nevus: a common type of skin growth that often appears as a small, dark brown spot.',
#     'BCC': 'Basal cell carcinoma: a type of skin cancer that most often develops on areas exposed to the sun.',
#     'AK': 'Actinic keratosis: a rough, scaly patch on the skin caused by years of sun exposure.',
#     'BKL': 'Benign keratosis: a non-cancerous skin condition that appears as a waxy brown, black, or tan growth.',
#     'DF': 'Dermatofibroma: a common growth on the skin, usually found on the lower legs, that can be pink, red, or brown.',
#     'VASC': 'Vascular lesion: a type of abnormal growth or mark on the skin that is made up of blood vessels.',
#     'SCC': 'Squamous cell carcinoma: a common form of skin cancer that develops in the squamous cells.',
# }

# disclaimer = """
# **Disclaimer:** This tool is intended for educational and entertainment purposes only and should not be used as a substitute for professional medical advice, diagnosis, or treatment. Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition. Remember, this AI is not a medical doctor, and its assessments are not diagnoses. Use this tool responsibly and always consult with a healthcare professional for any medical concerns.
# """

# def preprocess_image(image):
#     if image.mode != 'RGB':
#         image = image.convert('RGB')
#     image = np.array(image)  # Convert PIL image to numpy array
#     image = tf.image.resize(image, [224, 224])  # Resize to model expected dimensions
#     image = image / 255.0  # Normalize pixel values
#     image = np.expand_dims(image, axis=0)  # Add batch dimension for model prediction
#     return image

# def predict_and_explain(image, age, sex, location):
#     # Preprocess the image
#     image = preprocess_image(image)

#     # Prepare metadata
#     sex_encoded = [1 if sex == 'Male' else 0, 1 if sex == 'Female' else 0, 1 if sex == 'Other' else 0]
#     location_encoded = [1 if location == loc else 0 for loc in ['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other']]
#     default_values = [0] * (13 - 1 - len(sex_encoded) - len(location_encoded))
#     metadata = np.array([[age] + sex_encoded + location_encoded + default_values])

#     # Predict with both models
#     teacher_prediction = teacher_model.predict([image, metadata])[0]
#     student_prediction = student_model.predict([image, metadata])[0]

#     # Average the predictions if models perform the same
#     average_prediction = (teacher_prediction + student_prediction) / 2

#     # Suppose the teacher model is generally more reliable
#     # combined_predictions = 0.7 * teacher_predictions + 0.3 * student_predictions

#     # Weighted Prediction method from both models
#     # teacher_predictions = teacher_model.predict(test_input)
#     # student_predictions = student_model.predict(test_input)
#     # Weighted average of predictions
#     # weights = [0.7, 0.3]  # Adjust weights as necessary
#     # combined_predictions = weights[0] * teacher_predictions + weights[1] * student_predictions
#     # Decode the final prediction
#     # final_prediction = np.argmax(combined_predictions, axis=1)
#     # print("Final Prediction:", final_prediction)

#     predicted_class = np.argmax(average_prediction)
#     class_labels = list(diagnosis_descriptions.keys())
#     predicted_label = class_labels[predicted_class]

#     # Build the response
#     description = diagnosis_descriptions[predicted_label]
#     response = f"Predicted Lesion Type: {predicted_label}\nDescription: {description}"

#     return response

# iface = gr.Interface(
#     fn=predict_and_explain,
#     inputs=[
#         gr.Image(image_mode='RGB', sources=["upload", "webcam"], type='pil'),
#         gr.Number(label="Age"),
#         gr.Radio(choices=['Male', 'Female', 'Other'], label="Sex"),
#         gr.Dropdown(choices=['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other'], label="Anatomical Site")
#     ],
#     outputs="text",
#     title="Skin Lesion Classifier",
#     description="Upload an image of a skin lesion and enter metadata to predict its type. The results provide a preliminary classification and are not a substitute for professional medical advice."
# )

# iface.launch(debug=True)


our original working model below

In [ ]:
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model('/content/best_model.h5')

# Dictionary of diagnosis descriptions
diagnosis_descriptions = {
    'MEL': 'Melanoma: a serious form of skin cancer that begins in cells known as melanocytes.',
    'NV': 'Melanocytic nevus: a common type of skin growth that often appears as a small, dark brown spot.',
    'BCC': 'Basal cell carcinoma: a type of skin cancer that most often develops on areas exposed to the sun.',
    'AK': 'Actinic keratosis: a rough, scaly patch on the skin caused by years of sun exposure.',
    'BKL': 'Benign keratosis: a non-cancerous skin condition that appears as a waxy brown, black, or tan growth.',
    'DF': 'Dermatofibroma: a common growth on the skin, usually found on the lower legs, that can be pink, red, or brown.',
    'VASC': 'Vascular lesion: a type of abnormal growth or mark on the skin that is made up of blood vessels.',
    'SCC': 'Squamous cell carcinoma: a common form of skin cancer that develops in the squamous cells.',
    'UNK': 'None of the others: the lesion does not fit into any of the other categories.'
}

def preprocess_image(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image = np.array(image)  # Convert PIL image to numpy array
    image = tf.image.resize(image, [224, 224])  # Resize to model expected dimensions
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension for model prediction
    return image

def predict_and_explain(image, age, sex, location):
    # Preprocess the image
    image = preprocess_image(image)

    # Prepare metadata
    sex_encoded = [1 if sex == 'Male' else 0, 1 if sex == 'Female' else 0, 1 if sex == 'Other' else 0]
    location_encoded = [1 if location == loc else 0 for loc in ['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other']]
    default_values = [0] * (13 - 1 - len(sex_encoded) - len(location_encoded))
    metadata = np.array([[age] + sex_encoded + location_encoded + default_values])

    # Predict with model
    prediction = model.predict([image, metadata])[0]
    predicted_class = np.argmax(prediction)
    class_labels = list(diagnosis_descriptions.keys())
    predicted_label = class_labels[predicted_class]

    # Build the response
    description = diagnosis_descriptions[predicted_label]
    response = f"Predicted Lesion Type: {predicted_label}\nDescription: {description}"

    return response

iface = gr.Interface(
    fn=predict_and_explain,
    inputs=[
        gr.Image(image_mode='RGB', sources=["upload", "webcam"], type='pil'),  # Configured for image uploads and webcam captures
        gr.Number(label="Age"),
        gr.Radio(choices=['Male', 'Female', 'Other'], label="Sex"),
        gr.Dropdown(choices=['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other'], label="Anatomical Site")
    ],
    outputs="text",
    title="Skin Lesion Classifier",
    description="Upload an image of a skin lesion and enter metadata to predict its type. The results provide a preliminary classification and are not a substitute for professional medical advice."
)

iface.launch(debug=True)
